In [1]:
import re

import requests
from lxml import html

from neos_setup import extract_data_from_source, RAW_DATA_PATH, DATA_SOURCE_MAIN_PAGE_URL, write_log

def get_current_neos_page(main_page_url: str) -> int:
    current_neos_number = 0
    response = requests.get(url=main_page_url, timeout=10)

    if not response:
        write_log(message=f"Failed to connect to page {main_page_url}")

    else:
        write_log(message=f"Page responded with {response.status_code} status code")

        page_tree = html.fromstring(response.text)
        current_neos: str = page_tree.xpath("//p[@align='center']/b/text()")[0]
        current_neos_number: int = int(re.sub('\D', '', current_neos))

        print(f"Current number of known NEOs: {current_neos_number}")
        write_log(message=f"Current number of known NEOs: {current_neos_number}")

    return current_neos_number

In [3]:
from database_setup import SessionLocal, NEO

session = SessionLocal()

file_data = extract_data_from_source(RAW_DATA_PATH)
if file_data:
    print(file_data[:2])
    neos = [NEO(**neo) for neo in file_data]
    print(neos[:2])
    session.add_all(neos)
    session.commit()


[{'Name': '433', 'Epoch_MJD': 60000.0, 'SemMajAxis_AU': 1.4581291370906075, 'Ecc': 0.2227818987391125, 'Incl_deg': 10.827823537332183, 'LongAscNode_deg': 304.287035255947, 'ArgP_deg': 178.9270025329854, 'Mean_Anom_deg': 110.7776496890265, 'AbsMag': 10.82, 'SlopeParamG': 0.46}, {'Name': '719', 'Epoch_MJD': 60000.0, 'SemMajAxis_AU': 2.6371753293766673, 'Ecc': 0.5470449451219556, 'Incl_deg': 11.5760111039842, 'LongAscNode_deg': 183.8569961111151, 'ArgP_deg': 156.25035711115578, 'Mean_Anom_deg': 10.235964379787434, 'AbsMag': 15.53, 'SlopeParamG': 0.15}]
[<database_setup.NEO object at 0x7fe8e028d350>, <database_setup.NEO object at 0x7fe8e04001d0>]


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: neo.Name
[SQL: INSERT INTO neo ("Name", "Epoch_MJD", "SemMajAxis_AU", "Ecc", "Incl_deg", "LongAscNode_deg", "ArgP_deg", "Mean_Anom_deg", "AbsMag", "SlopeParamG") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: ('433', 60000.0, 1.4581291370906075, 0.2227818987391125, 10.827823537332183, 304.287035255947, 178.9270025329854, 110.7776496890265, 10.82, 0.46)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)